In [12]:
import requests
from bs4 import BeautifulSoup as bs
import re


def non_google_links(href):
    article = re.compile('http')
    google = re.compile('google')
    if href == None:
        return False
    elif article.search(href) and not google.search(href):
        return True
    else:
        return False

def google_top_results(url, n):
    page = requests.get(url)
    soup = bs(page.content, 'lxml')
    main = soup.find(id='main')
    links = main.find_all(href=non_google_links)
    news = []
    for link in links:
        clean_link = link.get('href')
        regex = re.compile(r'(http[s]?://.*?)&sa')
        m = regex.search(clean_link)
        news.append(m.group(1))
    news = list(set(news))
    return news[:n]

url = 'https://www.google.com/search?q=chicago&tbm=nws'
google_top_results(url, 10)




['https://chicago.suntimes.com/2020/8/14/21368202/calumet-river-army-corps-engineers-chicago-southeast-side-park-environment',
 'https://www.foxnews.com/media/chicago-store-owner-is-looted-twice-i-dont-know-what-i-am-going-to-do',
 'https://www.foxnews.com/us/chicago-democrat-kim-foxx-looting-riots-raymond-lopez',
 'https://www.chicagotribune.com/news/breaking/ct-chicago-looting-court-20200814-hv7kdbzlc5hmdmjkabaebu2yeq-story.html',
 'https://patch.com/illinois/chicago/brief-tornado-hits-rogers-park-derecho-blasts-chicago',
 'https://www.nbcchicago.com/news/local/chicago-protest-aims-to-shut-down-dan-ryan-expressway-this-weekend/2321821/',
 'https://abc7chicago.com/chicago-looting-police-mayor-lori-lightfoot-cpd/6370260/',
 'https://wgntv.com/news/chicago-news/police-say-theyre-preparing-for-weekend-to-avoid-repeat-of-downtown-looting/',
 'https://nypost.com/2020/08/13/pushback-in-chicago-as-blacks-are-sick-and-tired-of-looters-anarchists/',
 'https://www.chicagotribune.com/coronavirus

In [22]:
import pprint

page = requests.get('https://www.google.com/search?q=chicago&tbm=nws')
soup = bs(page.content, 'lxml')
main = soup.find(id='main')

links = []
for link in main.find_all('a'):
    if link.has_attr('href'):
        links.append(link['href'])
google_status = []
for link in links:
    google_status.append(non_google_links(link))

# pprint.pprint(links)
# print(google_status)
result =dict(zip(links,google_status))
result
links
sum(non_google_links(links))

TypeError: expected string or bytes-like object

In [26]:
url = 'https://www.google.com/search?q=chicago&tbm=nws'
page = requests.get(url)
page.status_code

200